https://github.com/ELS-RD/kernl/blob/main/tutorial/1%20-%20tiled%20matmul.ipynb

In [10]:
import torch

M, N, K = 6, 9, 4

# A = torch.rand((M, K))
# B = torch.rand((K, N))

A = torch.randint(0,50,(M, K))
B = torch.randint(0,50,(K, N))

In [11]:
print(A)
print(A.shape)
print()
print(B)
print(B.shape)

tensor([[18, 48, 27, 48],
        [ 6, 16, 33,  8],
        [17, 27, 31, 26],
        [ 3,  0, 26, 33],
        [11,  1, 35, 34],
        [14, 28, 28, 20]])
torch.Size([6, 4])

tensor([[ 5,  6, 28,  2, 44, 34,  1, 47,  2],
        [40, 23, 18,  1,  7, 45, 20, 38, 33],
        [15, 47, 38,  2, 30, 26, 33,  0, 19],
        [11, 14, 33, 47,  5,  3, 26, 18, 42]])
torch.Size([4, 9])


In [13]:
# for simplification tile shapes are all multiple of matrix shapes
# otherwise we would need to check matrix bounds and mask out of bounds values by 0s in tiles
block_M, block_N, block_K = M // 3, N // 3, K // 2
print(block_M, block_N, block_K)

output = torch.zeros((M, N))
total_load = 0
total_write = 0

for index_M in range(0, M, block_M):
    print("index_M ", index_M)
    start_M = index_M
    end_M = index_M + block_M
    print("end_M ", end_M)

    for index_N in range(0, N, block_N):
        print("index_N ", index_N)
        start_N = index_N
        end_N = index_N + block_N
        print("end_N ", end_N)
        accumulator = torch.zeros((block_M, block_N))
        for index_K in range(0, K, block_K):
            print("index_K ", index_K)
            start_K = index_K
            end_K = index_K + block_K
            print("end_K ", end_K)
            tile_A = A[start_M:end_M, start_K:end_K]
            print("tile_A SHAPE:: ", tile_A.shape)
            print("tile_A:: ", tile_A)
            total_load += tile_A.numel()
            print("total_load:: ", total_load)
            tile_B = B[start_K:end_K, start_N:end_N]
            print("tile_B SHAPE:: ", tile_B.shape)
            print("tile_B:: ", tile_B)
            total_load += tile_B.numel()
            print("total_load:: ", total_load)
            # @ means matmul in numpy and pytorch
            accumulator += tile_A @ tile_B
            print("accumulator SHAPE:: ", accumulator.shape)
            print("accumulator:: ", accumulator)
            print("---"*25)
            break
        output[start_M:end_M, start_N:end_N] = accumulator
        total_write += accumulator.numel()

# assert torch.allclose(output, A @ B)
# print("total load from GM:", total_load)
# print("total write to GM:", total_write)

2 3 2
index_M  0
end_M  2
index_N  0
end_N  3
index_K  0
end_K  2
tile_A SHAPE::  torch.Size([2, 2])
tile_A::  tensor([[18, 48],
        [ 6, 16]])
total_load::  4
tile_B SHAPE::  torch.Size([2, 3])
tile_B::  tensor([[ 5,  6, 28],
        [40, 23, 18]])
total_load::  10
accumulator SHAPE::  torch.Size([2, 3])
accumulator::  tensor([[2010., 1212., 1368.],
        [ 670.,  404.,  456.]])
---------------------------------------------------------------------------
index_N  3
end_N  6
index_K  0
end_K  2
tile_A SHAPE::  torch.Size([2, 2])
tile_A::  tensor([[18, 48],
        [ 6, 16]])
total_load::  14
tile_B SHAPE::  torch.Size([2, 3])
tile_B::  tensor([[ 2, 44, 34],
        [ 1,  7, 45]])
total_load::  20
accumulator SHAPE::  torch.Size([2, 3])
accumulator::  tensor([[  84., 1128., 2772.],
        [  28.,  376.,  924.]])
---------------------------------------------------------------------------
index_N  6
end_N  9
index_K  0
end_K  2
tile_A SHAPE::  torch.Size([2, 2])
tile_A::  tensor([[1